In [ ]:
#create a custom Rasa NLU component that leverages a Hugging Face model for intent classification, a core component of a Rasa-based dialogue system.

In [ ]:
# This is a Rasa custom NLU component that integrates with Hugging Face Transformers
# The code below should be placed in a custom Python file (e.g., hf_nlu.py)
# and configured in your Rasa pipeline in config.yml.

import logging
from typing import Any, Text, Dict, List, Type

from rasa.nlu.components import Component
from rasa.nlu.classifiers.classifier import IntentClassifier
from rasa.nlu.training_data import Message, TrainingData
from rasa.nlu.model import Metadata

from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification
import tensorflow as tf
import numpy as np

logger = logging.getLogger(__name__)

class HuggingFaceIntentClassifier(IntentClassifier, Component):
    """A custom intent classifier using Hugging Face's DistilBERT."""

    # Name of the component
    name = "HuggingFaceIntentClassifier"

    # Defines the required components in the pipeline before this one
    requires = []

    # Defines the components that can follow this one
    provides = ["intent", "intent_ranking"]

    # Defines the defaults for the configuration
    defaults = {
        "model_name": "distilbert-base-uncased",
        "label_mapping": {},
        "epochs": 5,
        "batch_size": 16
    }

    # Initializes the component
    def __init__(self, component_config: Dict[Text, Any] = None) -> None:
        super(HuggingFaceIntentClassifier, self).__init__(component_config)
        self.tokenizer = None
        self.model = None
        self.label_mapping = None

    @classmethod
    def create(cls, component_config: Dict[Text, Any], config: Dict[Text, Any]) -> "HuggingFaceIntentClassifier":
        """Creates a new instance of the component."""
        return cls(component_config)

    def train(
        self, training_data: TrainingData, config: Dict[Text, Any], **kwargs: Any
    ) -> None:
        """Train this component on a given training data."""
        # Get the unique intents from the training data
        intents = [e.data["intent"] for e in training_data.intent_examples]
        unique_intents = sorted(list(set(intents)))
        self.label_mapping = {intent: i for i, intent in enumerate(unique_intents)}
        
        # Load the pre-trained DistilBERT tokenizer and model
        self.tokenizer = DistilBertTokenizer.from_pretrained(self.component_config["model_name"])
        self.model = TFDistilBertForSequenceClassification.from_pretrained(
            self.component_config["model_name"], num_labels=len(unique_intents)
        )

        # Prepare training data
        texts = [e.text for e in training_data.intent_examples]
        labels = [self.label_mapping[e.data["intent"]] for e in training_data.intent_examples]

        # Tokenize the texts and convert to TensorFlow dataset
        tokenized_texts = self.tokenizer(texts, padding=True, truncation=True, return_tensors="tf")
        dataset = tf.data.Dataset.from_tensor_slices(
            (
                dict(tokenized_texts),
                labels
            )
        ).shuffle(len(texts)).batch(self.component_config["batch_size"])

        # Compile and train the model
        optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
        self.model.compile(optimizer=optimizer, loss=self.model.compute_loss)

        logger.info(f"Training DistilBERT model for {self.component_config['epochs']} epochs...")
        self.model.fit(dataset, epochs=self.component_config["epochs"])

    def process(self, message: Message, **kwargs: Any) -> None:
        """Process an incoming message and classify its intent."""
        if not self.model or not self.tokenizer or not self.label_mapping:
            logger.warning("Model not trained. Skipping intent classification.")
            return

        # Tokenize the input message
        inputs = self.tokenizer(
            [message.text], 
            padding=True, 
            truncation=True, 
            return_tensors="tf"
        )
        
        # Get predictions from the model
        predictions = self.model(inputs)
        logits = predictions.logits
        
        # Get the intent with the highest probability
        predicted_label = tf.argmax(logits, axis=1).numpy()[0]
        confidence = tf.nn.softmax(logits, axis=1).numpy()[0]
        
        # Map the label back to the original intent name
        intent_name = list(self.label_mapping.keys())[list(self.label_mapping.values()).index(predicted_label)]
        
        # Create a ranking of all intents
        intent_ranking = [
            {"name": intent, "confidence": float(confidence[self.label_mapping[intent]])}
            for intent in self.label_mapping.keys()
        ]
        intent_ranking = sorted(intent_ranking, key=lambda x: x["confidence"], reverse=True)

        # Update the message with the classified intent and ranking
        message.set("intent", {"name": intent_name, "confidence": float(confidence[predicted_label])}, add_to_output=True)
        message.set("intent_ranking", intent_ranking, add_to_output=True)

    def persist(self, file_name: Text, model_dir: Text) -> Dict[Text, Any]:
        """Persist this model into the passed directory."""
        # Save the tokenizer and model
        save_path = os.path.join(model_dir, file_name)
        self.model.save_pretrained(save_path)
        self.tokenizer.save_pretrained(save_path)
        
        # Save the label mapping as well
        with open(os.path.join(save_path, "label_mapping.json"), "w") as f:
            json.dump(self.label_mapping, f)

        # Return a dictionary with the filename and a reference to the saved files
        return {"file": file_name, "label_mapping_file": "label_mapping.json"}

    @classmethod
    def load(
        cls,
        meta: Dict[Text, Any],
        model_dir: Text,
        component_config: Dict[Text, Any],
        **kwargs: Any
    ) -> "HuggingFaceIntentClassifier":
        """Load a trained model from the storage."""
        # Load the tokenizer and model
        save_path = os.path.join(model_dir, meta["file"])
        tokenizer = DistilBertTokenizer.from_pretrained(save_path)
        model = TFDistilBertForSequenceClassification.from_pretrained(save_path)
        
        # Load the label mapping
        with open(os.path.join(save_path, meta["label_mapping_file"]), "r") as f:
            label_mapping = json.load(f)

        # Create and return the component instance
        component = cls(component_config)
        component.tokenizer = tokenizer
        component.model = model
        component.label_mapping = label_mapping
        return component



In [ ]:
#To use the custom Hugging Face intent classifier for inference in Rasa, follow these steps after implementing the component code in above referenced git code (e.g., in hf_nlu.py):

# config.yml 

language: en

pipeline:
  - name: WhitespaceTokenizer
  - name: CountVectorsFeaturizer
  - name: custom_components.hf_nlu.HuggingFaceIntentClassifier
    model_name: "distilbert-base-uncased"
    epochs: 5
    batch_size: 16


#Make sure the import path (e.g., custom_components.hf_nlu.HuggingFaceIntentClassifier) matches your project structure.


In [ ]:
rasa train


In [ ]:
#Step4: Run the Rasa Shell for Inference
rasa shell


In [ ]:
#Step 5:  Programmatic Inference
#If you want to use the model outside of Rasa (e.g., for batch inference or testing), you can do something like this:
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification
import tensorflow as tf
import json
import os

# Load model, tokenizer, and label mapping
MODEL_DIR = "models/nlu/custom_component"  # Adjust based on persist location

tokenizer = DistilBertTokenizer.from_pretrained(MODEL_DIR)
model = TFDistilBertForSequenceClassification.from_pretrained(MODEL_DIR)

with open(os.path.join(MODEL_DIR, "label_mapping.json")) as f:
    label_mapping = json.load(f)

inv_label_mapping = {v: k for k, v in label_mapping.items()}

# Inference function
def classify_intent(text):
    inputs = tokenizer([text], return_tensors="tf", truncation=True, padding=True)
    logits = model(inputs).logits
    probs = tf.nn.softmax(logits, axis=1).numpy()[0]
    predicted_label = tf.argmax(logits, axis=1).numpy()[0]
    return {
        "intent": inv_label_mapping[predicted_label],
        "confidence": float(probs[predicted_label]),
        "ranking": [
            {"intent": inv_label_mapping[i], "confidence": float(conf)}
            for i, conf in sorted(enumerate(probs), key=lambda x: x[1], reverse=True)
        ]
    }


In [ ]:
print(classify_intent("I want to book a flight"))

#output


#
# The classifier modifies the Message object with two keys: intent and intent_ranking.
{
  "intent": {
    "name": "book_flight",
    "confidence": 0.92},
  "intent_ranking": [
    {"name": "book_flight",
      "confidence": 0.92},
    {"name": "inform_location",
      "confidence": 0.05},
    {"name": "greet",
      "confidence": 0.02},
    {"name": "cancel",
      "confidence": 0.01} ]}
